# EMBS-BHI-2025: Robust and Reproducible AI Tutorial¶
Tutorial instructors: Ernest Namdar and Pascal Tyrrell

The dataset used in this tutorial is from BraTS 2025 Pediatrics.

## Evaluation Metrics for Pediatric BraTS Segmentation

3D segmentation keeps inter-slice context and typically delivers smoother, anatomically consistent predictions, yet 2D networks remain attractive because they are less complex to train, require fewer computational resources, and the manual ground truthing effort per slice is lower.

## Metric Taxonomy
- **Overlap-Based Metrics**
- **Distance-Based Metrics**
- **Detection / Object-Level Metrics**
- **Calibration / Probabilistic Metrics**
- **Region / Volume-Based Metrics**
- **Advanced / Specialized Metrics**
- **Multi-Class Extensions**
- **Composite Scores**

In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
from scipy.ndimage import label, binary_erosion
from scipy.spatial import cKDTree

try:
    import matplotlib.pyplot as plt
except ImportError:
    plt = None


In [5]:
PROJECT_ROOT = Path('.').resolve()
if not (PROJECT_ROOT / 'data').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_DIR = PROJECT_ROOT / 'data'
GROUND_TRUTH_DIR = DATA_DIR / 'GroundTruth'
INFERENCE_DIR = DATA_DIR / 'Inference'
RESULTS_DIR = PROJECT_ROOT / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f'Ground-truth masks: {GROUND_TRUTH_DIR}')
print(f'Inference masks:    {INFERENCE_DIR}')
print(f'Results directory:  {RESULTS_DIR}')


Ground-truth masks: /home/ernest/Desktop/Projects/EMBS_BHI_2025_Tutorial/Part2-Segmentation/data/GroundTruth
Inference masks:    /home/ernest/Desktop/Projects/EMBS_BHI_2025_Tutorial/Part2-Segmentation/data/Inference
Results directory:  /home/ernest/Desktop/Projects/EMBS_BHI_2025_Tutorial/Part2-Segmentation/results


In [7]:
def load_canonical_mask(path: Path):
    image = nib.load(str(path))
    canonical = nib.as_closest_canonical(image)
    data = canonical.get_fdata()
    mask = data > 0
    zooms = canonical.header.get_zooms()[:3]
    return mask.astype(bool), np.asarray(zooms, dtype=float)

def strip_nii_gz(name: str) -> str:
    return name[:-7] if name.endswith('.nii.gz') else Path(name).stem

def _surface_voxels(mask: np.ndarray):
    if not mask.any():
        return np.empty((0, mask.ndim), dtype=float)
    eroded = binary_erosion(mask, structure=np.ones((3, 3, 3)))
    surface = mask ^ eroded
    coords = np.argwhere(surface)
    return coords

def _voxel_to_world(coords: np.ndarray, spacing):
    return coords * spacing

def _surface_distance_arrays(gt_surface, pred_surface, spacing):
    if gt_surface.size == 0 and pred_surface.size == 0:
        return np.array([0.0]), np.array([0.0])
    if gt_surface.size == 0:
        return np.array([np.inf]), np.zeros(pred_surface.shape[0])
    if pred_surface.size == 0:
        return np.zeros(gt_surface.shape[0]), np.array([np.inf])
    gt_world = _voxel_to_world(gt_surface, spacing)
    pred_world = _voxel_to_world(pred_surface, spacing)

    gt_tree = cKDTree(gt_world)
    pred_tree = cKDTree(pred_world)

    d_gt_to_pred, _ = pred_tree.query(gt_world)
    d_pred_to_gt, _ = gt_tree.query(pred_world)

    return d_gt_to_pred, d_pred_to_gt


### Overlap-Based Metrics
Dice, Jaccard/IoU, and Relative Volume Difference quantify volumetric agreement. They are easy to interpret but insensitive to small boundary errors when volumes are large.

In [10]:
def dice_coefficient(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred).sum()
    volumes = y_true.sum() + y_pred.sum()
    if volumes == 0:
        return 1.0
    return 2.0 * intersection / volumes

def jaccard_index(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred).sum()
    union = np.logical_or(y_true, y_pred).sum()
    if union == 0:
        return 1.0
    return intersection / union

def relative_volume_difference(y_true, y_pred):
    gt_vol = y_true.sum()
    pred_vol = y_pred.sum()
    if gt_vol == 0:
        return np.inf if pred_vol > 0 else 0.0
    return (pred_vol - gt_vol) / gt_vol

def absolute_volume_difference(y_true, y_pred):
    return abs(y_pred.sum() - y_true.sum())


### Distance-Based Metrics
Hausdorff-style distances respond to worst-case boundary errors; percentile variants trim outliers. Surface Dice and NSD reward boundary agreement within a tolerance but depend on spacing.

In [13]:
def hausdorff_distance(y_true, y_pred, spacing):
    gt_surface = _surface_voxels(y_true)
    pred_surface = _surface_voxels(y_pred)
    d_gt_to_pred, d_pred_to_gt = _surface_distance_arrays(gt_surface, pred_surface, spacing)
    if np.isinf(d_gt_to_pred).any() or np.isinf(d_pred_to_gt).any():
        return np.inf
    return max(d_gt_to_pred.max(), d_pred_to_gt.max())

def percentile_hausdorff(y_true, y_pred, spacing, percentile=95):
    gt_surface = _surface_voxels(y_true)
    pred_surface = _surface_voxels(y_pred)
    d_gt_to_pred, d_pred_to_gt = _surface_distance_arrays(gt_surface, pred_surface, spacing)
    distances = np.concatenate([d_gt_to_pred, d_pred_to_gt])
    if np.isinf(distances).any():
        return np.inf
    return np.percentile(distances, percentile)

def mean_surface_distance(y_true, y_pred, spacing):
    gt_surface = _surface_voxels(y_true)
    pred_surface = _surface_voxels(y_pred)
    d_gt_to_pred, d_pred_to_gt = _surface_distance_arrays(gt_surface, pred_surface, spacing)
    distances = np.concatenate([d_gt_to_pred, d_pred_to_gt])
    if np.isinf(distances).any():
        return np.inf
    return distances.mean()

def normalized_surface_dice(y_true, y_pred, spacing, tolerance=1.0):
    gt_surface = _surface_voxels(y_true)
    pred_surface = _surface_voxels(y_pred)
    if gt_surface.size == 0 and pred_surface.size == 0:
        return 1.0
    d_gt_to_pred, d_pred_to_gt = _surface_distance_arrays(gt_surface, pred_surface, spacing)
    gt_within = (d_gt_to_pred <= tolerance).sum()
    pred_within = (d_pred_to_gt <= tolerance).sum()
    denom = gt_surface.shape[0] + pred_surface.shape[0]
    if denom == 0:
        return 0.0
    return (gt_within + pred_within) / denom

def surface_dice(y_true, y_pred, spacing, tolerance=1.0):
    gt_surface = _surface_voxels(y_true)
    pred_surface = _surface_voxels(y_pred)
    if gt_surface.size == 0 and pred_surface.size == 0:
        return 1.0
    d_gt_to_pred, d_pred_to_gt = _surface_distance_arrays(gt_surface, pred_surface, spacing)
    sensitivity = (d_gt_to_pred <= tolerance).sum() / max(gt_surface.shape[0], 1)
    precision = (d_pred_to_gt <= tolerance).sum() / max(pred_surface.shape[0], 1)
    if sensitivity + precision == 0:
        return 0.0
    return 2 * sensitivity * precision / (sensitivity + precision)


### Detection / Object-Level Metrics
Lesion-wise statistics look at connected components and highlight under/over-segmentation patterns that voxelwise scores can miss.

In [16]:
def lesionwise_stats(y_true, y_pred):
    labeled_true, num_true = label(y_true)
    labeled_pred, num_pred = label(y_pred)

    if num_true == 0:
        recall = 1.0 if num_pred == 0 else 0.0
    else:
        hits = 0
        for idx in range(1, num_true + 1):
            component = labeled_true == idx
            if np.logical_and(component, y_pred).any():
                hits += 1
        recall = hits / num_true

    if num_pred == 0:
        precision = 1.0 if num_true == 0 else 0.0
    else:
        hits = 0
        for idx in range(1, num_pred + 1):
            component = labeled_pred == idx
            if np.logical_and(component, y_true).any():
                hits += 1
        precision = hits / num_pred

    return recall, precision


### Region / Volume-Based Metrics
Absolute and relative volume differences are critical for downstream treatment planning, but they ignore shape fidelity.

### Advanced / Specialized Metrics
Mean Surface Distance (MSD) complements Hausdorff scores by averaging surface discrepancies; it is less sensitive to single outliers.

In [20]:
def compute_metrics(y_true, y_pred, spacing):
    metrics = {}
    metrics['dice'] = dice_coefficient(y_true, y_pred)
    metrics['jaccard'] = jaccard_index(y_true, y_pred)
    metrics['rvd'] = relative_volume_difference(y_true, y_pred)
    metrics['avd_voxels'] = absolute_volume_difference(y_true, y_pred)

    metrics['hausdorff'] = hausdorff_distance(y_true, y_pred, spacing)
    metrics['hausdorff95'] = percentile_hausdorff(y_true, y_pred, spacing, percentile=95)
    metrics['nsd'] = normalized_surface_dice(y_true, y_pred, spacing, tolerance=1.0)
    metrics['surface_dice'] = surface_dice(y_true, y_pred, spacing, tolerance=1.0)
    metrics['msd'] = mean_surface_distance(y_true, y_pred, spacing)

    recall, precision = lesionwise_stats(y_true, y_pred)
    metrics['lesion_recall'] = recall
    metrics['lesion_precision'] = precision

    return metrics


### Batch Evaluation Loop
The loop below iterates over every case, reports progress every 10 subjects, and saves the aggregated metrics for downstream analysis.

In [23]:
def collect_case_ids():
    gt_cases = {strip_nii_gz(p.name) for p in GROUND_TRUTH_DIR.glob('*.nii.gz')}
    pred_cases = {strip_nii_gz(p.name) for p in INFERENCE_DIR.glob('*.nii.gz')}
    common = sorted(gt_cases & pred_cases)
    missing_preds = sorted(gt_cases - pred_cases)
    missing_gts = sorted(pred_cases - gt_cases)
    if missing_preds:
        print(f'Missing inference for {len(missing_preds)} cases (first 5): {missing_preds[:5]}')
    if missing_gts:
        print(f'Missing ground truth for {len(missing_gts)} cases (first 5): {missing_gts[:5]}')
    return common

case_ids = collect_case_ids()
print(f'Total cases to process: {len(case_ids)}')


Total cases to process: 261


In [25]:
results = []
for idx, case_id in enumerate(case_ids, start=1):
    gt_mask, spacing = load_canonical_mask(GROUND_TRUTH_DIR / f'{case_id}.nii.gz')
    pred_mask, _ = load_canonical_mask(INFERENCE_DIR / f'{case_id}.nii.gz')

    metrics = compute_metrics(gt_mask, pred_mask, spacing)
    metrics['case_id'] = case_id
    metrics['voxel_spacing_x'] = spacing[0]
    metrics['voxel_spacing_y'] = spacing[1]
    metrics['voxel_spacing_z'] = spacing[2]
    results.append(metrics)

    if idx % 10 == 0 or idx == len(case_ids):
        print(f'Processed {idx} / {len(case_ids)} cases...')

metrics_df = pd.DataFrame(results)
output_csv = RESULTS_DIR / 'evaluation_metrics.csv'
metrics_df.to_csv(output_csv, index=False)
print(f'Saved metrics to {output_csv.relative_to(PROJECT_ROOT)}')
metrics_df.head()


Processed 10 / 261 cases...
Processed 20 / 261 cases...
Processed 30 / 261 cases...
Processed 40 / 261 cases...
Processed 50 / 261 cases...
Processed 60 / 261 cases...
Processed 70 / 261 cases...
Processed 80 / 261 cases...
Processed 90 / 261 cases...
Processed 100 / 261 cases...
Processed 110 / 261 cases...
Processed 120 / 261 cases...
Processed 130 / 261 cases...
Processed 140 / 261 cases...
Processed 150 / 261 cases...
Processed 160 / 261 cases...
Processed 170 / 261 cases...
Processed 180 / 261 cases...
Processed 190 / 261 cases...
Processed 200 / 261 cases...
Processed 210 / 261 cases...
Processed 220 / 261 cases...
Processed 230 / 261 cases...
Processed 240 / 261 cases...
Processed 250 / 261 cases...
Processed 260 / 261 cases...
Processed 261 / 261 cases...
Saved metrics to results/evaluation_metrics.csv


,dice,jaccard,rvd,avd_voxels,hausdorff,hausdorff95,nsd,surface_dice,msd,lesion_recall,lesion_precision,case_id,voxel_spacing_x,voxel_spacing_y,voxel_spacing_z
0,0.939661,0.886189,-0.001710,220,10.440307,1.414214,0.918363,0.919463,0.432630,0.111111,1.0,BraTS-PED-00001-000,1.0,1.0,1.0
1,0.974891,0.951011,-0.015203,266,4.000000,1.000000,0.995115,0.995155,0.159473,0.500000,1.0,BraTS-PED-00002-000,1.0,1.0,1.0
2,0.943192,0.892491,0.011384,189,84.693565,1.414214,0.931390,0.933062,0.480986,0.200000,1.0,BraTS-PED-00003-000,1.0,1.0,1.0
3,0.948302,0.901687,0.005383,844,40.422766,2.449490,0.819089,0.822027,0.715332,0.100000,1.0,BraTS-PED-00004-000,1.0,1.0,1.0
4,0.945409,0.896470,-0.002320,40,5.830952,1.414214,0.920215,0.921760,0.416009,0.250000,1.0,BraTS-PED-00005-000,1.0,1.0,1.0


Calibration, multi-class, and composite scoring layers can be added on top of these voxel-level metrics once probabilistic outputs or subregion annotations are available.

BraTS remains a multi-subregion benchmark; although we focus on whole-tumor masks here, the same evaluation scaffolding extends naturally to per-label assessments.